In [1]:
from utils.dataloader import *
from utils.sift import *
from utils.helper import *
from utils.classifier import get_report, down_sampling, vote, up_sampling, one_vs_other_up_sampling, distance_vote, grid_search
import pickle


In [2]:
RANDOME_STATE= 66
ROOT_PATH, DATA_PATH, OUT_PATH = get_paths()
print(ROOT_PATH)

/Users/moooooo16/Documents/Computer Vision/Project/elpv


In [3]:
train_collection = test_collection = y_train = y_test = None


with open(os.path.join(OUT_PATH, 'sift_X.pkl'), 'rb') as f:
    (train_collection, test_collection) = pickle.load(f)


with open(os.path.join(OUT_PATH, 'sift_y.pkl'), 'rb') as f:
    (y_train, y_test) = pickle.load(f)


In [4]:
hog_X_train, hog_X_test = None, None

with open(os.path.join(OUT_PATH, 'hog_X.pkl'), 'rb') as f:
    (hog_X_train, hog_X_test) = pickle.load(f)

hog_X_test = np.array(hog_X_test)
hog_X_train = np.array(hog_X_train)
print(hog_X_test.shape, hog_X_train.shape)

(656, 9000) (1968, 9000)


In [5]:

ks = train_collection.keys()
y_train = np.array(y_train)
y_test = np.array(y_test)

for a, b in zip(train_collection.values(), test_collection.values()):
    print(a.shape, b.shape)

print(ks)
print(y_train.shape, y_test.shape)

(1968, 128) (656, 128)
(1968, 256) (656, 256)
(1968, 512) (656, 512)
(1968, 1024) (656, 1024)
dict_keys([128, 256, 512, 1024])
(1968,) (656,)


# Undersampling method

441
88
42
235
690
133
37
302

In [6]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

param = {
    'kernel': ['linear', 'rbf'],
    'C' : [0.01, 0.1, 1, 10, 100]
}


clf_collections = []
predictions_collections = []
votes_collections = []


In [6]:

logger = my_logger(os.path.join(OUT_PATH, 'logs/undersampling.log'), 'undersampling')
logger.warning(f'k,i,j,best_score,,best_clf,mean_train_score,mean_test_score')

total_classes = np.unique(y_train)

for k in ks:
    
    X_train = np.array(train_collection[k])
    X_test = np.array(test_collection[k])
    predictions = []
    
    clfs = down_sampling(total_classes=total_classes, X_train=X_train, y_train=y_train, 
                         clf = SVC, 
                         param=param, k=k, logger=logger)


    predictions, votes, counts = vote(X_test=X_test, clfs=clfs, predictions=predictions)
    
    acc, conf_mat = get_report(y_test, votes)
    print()
    print(f'Overall Test Accuracy: {acc:.2f}')
    print(f'Confusion Matrix: \n{conf_mat}')
    print('-'*100)
    
    predictions_collections.append(predictions)
    votes_collections.append(votes)
    clf_collections.append(clfs)
    
    logger.handlers[0].flush()

            

Class: [0 1] Val Score: 0.74 use {'C': 0.1, 'kernel': 'linear'}
Class: [0 2] Val Score: 0.65 use {'C': 0.01, 'kernel': 'linear'}
Class: [0 3] Val Score: 0.86 use {'C': 10, 'kernel': 'rbf'}
Class: [0 4] Val Score: 1.00 use {'C': 10, 'kernel': 'rbf'}
Class: [0 5] Val Score: 1.00 use {'C': 0.01, 'kernel': 'linear'}
Class: [0 6] Val Score: 1.00 use {'C': 0.01, 'kernel': 'linear'}
Class: [0 7] Val Score: 1.00 use {'C': 100, 'kernel': 'rbf'}
Class: [1 2] Val Score: 0.73 use {'C': 0.1, 'kernel': 'rbf'}
Class: [1 3] Val Score: 0.85 use {'C': 10, 'kernel': 'rbf'}
Class: [1 4] Val Score: 0.99 use {'C': 10, 'kernel': 'rbf'}
Class: [1 5] Val Score: 0.99 use {'C': 0.1, 'kernel': 'rbf'}
Class: [1 6] Val Score: 1.00 use {'C': 1, 'kernel': 'rbf'}
Class: [1 7] Val Score: 0.99 use {'C': 0.1, 'kernel': 'rbf'}
Class: [2 3] Val Score: 0.78 use {'C': 0.01, 'kernel': 'linear'}
Class: [2 4] Val Score: 0.99 use {'C': 1, 'kernel': 'rbf'}
Class: [2 5] Val Score: 1.00 use {'C': 0.01, 'kernel': 'linear'}
Class: [2

# SMOTE Upsampling

In [7]:



logger = my_logger(os.path.join(OUT_PATH, 'logs/upsampling.log'), 'upsampling')
logger.warning(f'k,i,j,best_score,,best_clf,mean_train_score,mean_test_score')


total_classes = np.unique(y_train)
for k in ks:
    
    X_train = np.array(train_collection[k])
    X_test = np.array(test_collection[k])

    predictions = []
    
    clfs = up_sampling(total_classes=total_classes, X_train=X_train, y_train=y_train, 
                         clf = SVC,
                         param=param, k=k, logger=logger,
                         knn = KNeighborsClassifier)


    predictions, votes, counts = vote(X_test=X_test, clfs=clfs, predictions=predictions)
    
    acc, conf_mat = get_report(y_test, votes)
    print()
    print(f'Overall Test Accuracy: {acc:.2f}')
    print(f'Confusion Matrix: \n{conf_mat}')
    print('-'*100)
    
    predictions_collections.append(predictions)
    votes_collections.append(votes)
    clf_collections.append(clfs)
    
    logger.handlers[0].flush()

            

Generated 440 synthetic samples.
Class: [0. 1.] Val Score: 0.92 use {'C': 100, 'kernel': 'rbf'}
Generated 420 synthetic samples.
Class: [0. 2.] Val Score: 0.94 use {'C': 100, 'kernel': 'rbf'}
Generated 235 synthetic samples.
Class: [0. 3.] Val Score: 0.92 use {'C': 100, 'kernel': 'rbf'}
Generated 441 synthetic samples.
Class: [0. 4.] Val Score: 1.00 use {'C': 1, 'kernel': 'rbf'}
Generated 399 synthetic samples.
Class: [0. 5.] Val Score: 1.00 use {'C': 0.01, 'kernel': 'linear'}
Generated 407 synthetic samples.
Class: [0. 6.] Val Score: 1.00 use {'C': 0.01, 'kernel': 'linear'}
Generated 302 synthetic samples.
Class: [0. 7.] Val Score: 1.00 use {'C': 10, 'kernel': 'rbf'}
Generated 84 synthetic samples.
Class: [1. 2.] Val Score: 0.85 use {'C': 10, 'kernel': 'linear'}
Generated 176 synthetic samples.
Class: [1. 3.] Val Score: 0.93 use {'C': 100, 'kernel': 'rbf'}
Generated 616 synthetic samples.
Class: [1. 4.] Val Score: 1.00 use {'C': 1, 'kernel': 'rbf'}
Generated 88 synthetic samples.
Clas

In [6]:
logger = my_logger(os.path.join(OUT_PATH, 'logs/ova.log'), 'ova')
logger.warning(f'k,i,j,best_score,,best_clf,mean_train_score,mean_test_score')


total_classes = np.unique(y_train)
for k in ks:
    
    X_train = np.array(train_collection[k])
    X_test = np.array(test_collection[k])

    predictions = []
    
    clfs = one_vs_other_up_sampling(total_classes, X_train, y_train,
                                clf = SVC,
                                param=param, k=k, logger=logger,
                                knn = KNeighborsClassifier, n_neighbors=2)


    distance , pred = distance_vote(X_test, clfs, None)
    
    acc, conf_mat = get_report(y_test, pred)
    print()
    print(f'Overall Test Accuracy: {acc:.2f}')
    print(f'Confusion Matrix: \n{conf_mat}')
    print('-'*100)
    
    # predictions_collections.append(predictions)
    # votes_collections.append(votes)
    clf_collections.append(clfs)
    
    logger.handlers[0].flush()

Generated 1323 synthetic samples.
Trainig on: (array([-1.,  0.]), array([1527, 1764]))
Class: [-1.  0.] Val Score: 0.96 use {'C': 100, 'kernel': 'rbf'}
Generated 1848 synthetic samples.
Trainig on: (array([-1.,  1.]), array([1880, 1936]))
Class: [-1.  1.] Val Score: 0.97 use {'C': 100, 'kernel': 'rbf'}
Generated 1890 synthetic samples.
Trainig on: (array([-1.,  2.]), array([1926, 1932]))
Class: [-1.  2.] Val Score: 0.94 use {'C': 100, 'kernel': 'rbf'}
Generated 1645 synthetic samples.
Trainig on: (array([-1.,  3.]), array([1733, 1880]))
Class: [-1.  3.] Val Score: 0.97 use {'C': 100, 'kernel': 'rbf'}
Generated 690 synthetic samples.
Trainig on: (array([-1.,  4.]), array([1278, 1380]))
Class: [-1.  4.] Val Score: 0.92 use {'C': 100, 'kernel': 'rbf'}
Generated 1729 synthetic samples.
Trainig on: (array([-1.,  5.]), array([1835, 1862]))
Class: [-1.  5.] Val Score: 0.98 use {'C': 100, 'kernel': 'rbf'}
Generated 1924 synthetic samples.
Trainig on: (array([-1.,  6.]), array([1931, 1961]))
Cl

In [7]:
logger = my_logger(os.path.join(OUT_PATH, 'logs/hog_normal.log'), 'hog_normal')
logger.warning(f'best_score,,best_clf,mean_train_score,mean_test_score')


clf = SVC()

predictions = []

clf = grid_search(hog_X_train, y_train,
                  clf, param, scoring='accuracy', 
                  k = None, classes=None,
                  logger=logger)

pred = clf.predict(hog_X_test)

acc, conf_mat = get_report(y_test, pred)
print()
print(f'Overall Test Accuracy: {acc:.2f}')
print(f'Confusion Matrix: \n{conf_mat}')
print('-'*100)

logger.handlers[0].flush()

Class: [0 1 2 3 4 5 6 7] Val Score: 0.74 use {'C': 10, 'kernel': 'rbf'}
Testset Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.95      0.82       147
           1       0.50      0.28      0.36        29
           2       0.00      0.00      0.00        14
           3       0.89      0.71      0.79        78
           4       0.70      0.89      0.79       230
           5       0.60      0.13      0.22        45
           6       0.00      0.00      0.00        13
           7       0.66      0.57      0.61       100

    accuracy                           0.72       656
   macro avg       0.51      0.44      0.45       656
weighted avg       0.68      0.72      0.68       656



Overall Test Accuracy: 0.72
Confusion Matrix: 
[[139   4   0   4   0   0   0   0]
 [ 19   8   0   2   0   0   0   0]
 [ 11   2   0   1   0   0   0   0]
 [ 21   2   0  55   0   0   0   0]
 [  0   0   0   0 205   4   0  21]
 [  0   0   0   0  33 

In [7]:

logger = my_logger(os.path.join(OUT_PATH, 'logs/hog_undersampling.log'), 'hog_undersampling')
logger.warning(f'i,j,best_score,,best_clf,mean_train_score,mean_test_score')

total_classes = np.unique(y_train)

k = None
    
X_train = np.array(hog_X_train)
X_test = np.array(hog_X_test)
predictions = []

clfs = down_sampling(total_classes=total_classes, X_train=X_train, y_train=y_train, 
                        clf = SVC, 
                        param=param, k=k, logger=logger)


predictions, votes, counts = vote(X_test=X_test, clfs=clfs, predictions=predictions)

acc, conf_mat = get_report(y_test, votes)
print()
print(f'Overall Test Accuracy: {acc:.2f}')
print(f'Confusion Matrix: \n{conf_mat}')
print('-'*100)

predictions_collections.append(predictions)
votes_collections.append(votes)
clf_collections.append(clfs)

logger.handlers[0].flush()

            

Class: [0 1] Val Score: 0.69 use {'C': 10, 'kernel': 'rbf'}
Class: [0 2] Val Score: 0.68 use {'C': 10, 'kernel': 'rbf'}
Class: [0 3] Val Score: 0.84 use {'C': 10, 'kernel': 'rbf'}
Class: [0 4] Val Score: 1.00 use {'C': 0.01, 'kernel': 'linear'}
Class: [0 5] Val Score: 1.00 use {'C': 0.01, 'kernel': 'linear'}
Class: [0 6] Val Score: 1.00 use {'C': 0.01, 'kernel': 'linear'}
Class: [0 7] Val Score: 1.00 use {'C': 0.01, 'kernel': 'linear'}
Class: [1 2] Val Score: 0.65 use {'C': 1, 'kernel': 'rbf'}
Class: [1 3] Val Score: 0.85 use {'C': 1, 'kernel': 'rbf'}
Class: [1 4] Val Score: 1.00 use {'C': 0.01, 'kernel': 'linear'}
Class: [1 5] Val Score: 1.00 use {'C': 0.01, 'kernel': 'linear'}
Class: [1 6] Val Score: 1.00 use {'C': 0.01, 'kernel': 'linear'}
Class: [1 7] Val Score: 1.00 use {'C': 0.01, 'kernel': 'linear'}
Class: [2 3] Val Score: 0.83 use {'C': 0.01, 'kernel': 'linear'}
Class: [2 4] Val Score: 1.00 use {'C': 0.01, 'kernel': 'linear'}
Class: [2 5] Val Score: 1.00 use {'C': 0.01, 'kernel

In [8]:
logger = my_logger(os.path.join(OUT_PATH, 'logs/hog_ova.log'), 'hog_ova')
logger.warning(f'i,j,best_score,,best_clf,mean_train_score,mean_test_score')


total_classes = np.unique(y_train)
k = None

X_train = np.array(hog_X_train)
X_test = np.array(hog_X_test)

predictions = []

clfs = one_vs_other_up_sampling(total_classes, X_train, y_train,
                            clf = SVC,
                            param=param, k=k, logger=logger,
                            knn = KNeighborsClassifier, n_neighbors=2)


distance , pred = distance_vote(X_test, clfs, None)

acc, conf_mat = get_report(y_test, pred)
print()
print(f'Overall Test Accuracy: {acc:.2f}')
print(f'Confusion Matrix: \n{conf_mat}')
print('-'*100)

# predictions_collections.append(predictions)
# votes_collections.append(votes)
clf_collections.append(clfs)

logger.handlers[0].flush()

Generated 1323 synthetic samples.
Trainig on: (array([-1.,  0.]), array([1527, 1764]))
Class: [-1.  0.] Val Score: 0.96 use {'C': 10, 'kernel': 'rbf'}
Generated 1848 synthetic samples.
Trainig on: (array([-1.,  1.]), array([1880, 1936]))
Class: [-1.  1.] Val Score: 0.99 use {'C': 10, 'kernel': 'rbf'}
Generated 1890 synthetic samples.
Trainig on: (array([-1.,  2.]), array([1926, 1932]))
Class: [-1.  2.] Val Score: 0.99 use {'C': 10, 'kernel': 'rbf'}
Generated 1645 synthetic samples.
Trainig on: (array([-1.,  3.]), array([1733, 1880]))
Class: [-1.  3.] Val Score: 0.99 use {'C': 10, 'kernel': 'rbf'}
Generated 690 synthetic samples.
Trainig on: (array([-1.,  4.]), array([1278, 1380]))
Class: [-1.  4.] Val Score: 0.92 use {'C': 0.1, 'kernel': 'linear'}
Generated 1729 synthetic samples.
Trainig on: (array([-1.,  5.]), array([1835, 1862]))
Class: [-1.  5.] Val Score: 0.99 use {'C': 10, 'kernel': 'rbf'}
Generated 1924 synthetic samples.
Trainig on: (array([-1.,  6.]), array([1931, 1961]))
Clas

In [7]:
logger = my_logger(os.path.join(OUT_PATH, 'logs/hog_ova.log'), 'hog_ova')
logger.warning(f'i,j,best_score,,best_clf,mean_train_score,mean_test_score')



X_train = np.array(hog_X_train)
X_test = np.array(hog_X_test)
y_train = np.array([y if y < 4 else y -4 for y in y_train])
y_test = np.array([y if y < 4 else y -4 for y in y_test])

total_classes = np.unique(y_train)
k = None

predictions = []

clfs = one_vs_other_up_sampling(total_classes, X_train, y_train,
                            clf = SVC,
                            param=param, k=k, logger=logger,
                            knn = KNeighborsClassifier, n_neighbors=2)


distance , pred = distance_vote(X_test, clfs, None)

acc, conf_mat = get_report(y_test, pred)
print()
print(f'Overall Test Accuracy: {acc:.2f}')
print(f'Confusion Matrix: \n{conf_mat}')
print('-'*100)

# predictions_collections.append(predictions)
# votes_collections.append(votes)
clf_collections.append(clfs)

logger.handlers[0].flush()

Trainig on: (array([-1,  0]), array([ 837, 1131]))
Class: [-1  0] Val Score: 0.77 use {'C': 10, 'kernel': 'rbf'}
Generated 1547 synthetic samples.
Trainig on: (array([-1.,  1.]), array([1747, 1768]))
Class: [-1.  1.] Val Score: 0.99 use {'C': 10, 'kernel': 'rbf'}
Generated 1817 synthetic samples.
Trainig on: (array([-1.,  2.]), array([1889, 1896]))
Class: [-1.  2.] Val Score: 0.99 use {'C': 10, 'kernel': 'rbf'}
Generated 1074 synthetic samples.
Trainig on: (array([-1.,  3.]), array([1431, 1611]))
Class: [-1.  3.] Val Score: 0.96 use {'C': 10, 'kernel': 'rbf'}
Testset Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.87      0.80       377
           1       0.57      0.23      0.33        74
           2       0.00      0.00      0.00        27
           3       0.68      0.68      0.68       178

    accuracy                           0.71       656
   macro avg       0.50      0.45      0.45       656
weighted avg       0.67